 Smart Garbage Collection System (Python – OOP Based)

Global Billing Catalog

In [1]:
BILLING_INFO = {
    "biodegradable": 0,
    "recyclable": 2,
    "non_recyclable": 5
}


Garbage Item Class

In [2]:
class Garbage:
    def __init__(self, garbage_type, sub_type=None):
        """
        garbage_type: biodegradable / non_biodegradable
        sub_type: recyclable / non_recyclable (only for non_biodegradable)
        """
        self.garbage_type = garbage_type
        self.sub_type = sub_type


User Class (Login + Warning + Billing)

In [3]:
class User:
    def __init__(self, username, password, name, address):
        self.username = username
        self.password = password
        self.name = name
        self.address = address
        self.billing_catalog = {
            "biodegradable": 0,
            "recyclable": 0,
            "non_recyclable": 0
        }

    def receive_warning(self, message):
        print(f"[WARNING for {self.name}] {message}")

    def check_bill(self):
        total = 0
        print(f"\n--- Billing Summary for {self.name} ---")
        for k, v in self.billing_catalog.items():
            cost = v * BILLING_INFO[k]
            print(f"{k}: {v} items -> {cost} tk")
            total += cost
        print(f"TOTAL BILL: {total} tk\n")


Base Bin Class (Inheritance)

In [4]:
class Bin:
    def __init__(self, capacity=None):
        self.capacity = capacity
        self.items = []

    def is_full(self):
        if self.capacity is None:
            return False
        return len(self.items) >= self.capacity


Source Bin

In [5]:
class SourceBin(Bin):
    def __init__(self, user, gmp_bin, capacity=10):
        super().__init__(capacity)
        self.user = user
        self.gmp_bin = gmp_bin

    def receive_garbage(self, garbage):
        if self.is_full():
            self.send_warning("Reduce garbage production")
        else:
            self.items.append(garbage)
            self.send_to_gmp()

    def send_to_gmp(self):
        if self.items:
            item = self.items.pop(0)
            self.gmp_bin.receive_garbage(item, self.user)

    def send_warning(self, message):
        self.user.receive_warning(message)

    def receive_warning(self, message):
        self.user.receive_warning("Reduce garbage production")


GMP Bin

In [6]:
class GMPBin(Bin):
    def __init__(self, b_bin, nb_bin, capacity=10):
        super().__init__(capacity)
        self.b_bin = b_bin
        self.nb_bin = nb_bin

    def receive_garbage(self, garbage, user):
        if self.is_full():
            self.send_warning()
        else:
            self.items.append(garbage)
            self.allocate(garbage, user)
            self.items.pop(0)

    def allocate(self, garbage, user):
        if garbage.garbage_type == "biodegradable":
            self.b_bin.receive_garbage(garbage)
            user.billing_catalog["biodegradable"] += 1
        else:
            self.nb_bin.receive_garbage(garbage, user)

    def send_warning(self):
        print("[GMP-bin] Warning from GMP-bin")

    def receive_warning(self, message):
        print("[GMP-bin] Warning from GMP-bin")


Biodegradable Bin

In [7]:
class BBin(Bin):
    def __init__(self):
        super().__init__(capacity=None)

    def receive_garbage(self, garbage):
        self.items.append(garbage)
        print("Biodegradable garbage sent to plant")


Non-Biodegradable Bin

In [8]:
class NBBin(Bin):
    def __init__(self, r_bin, nr_bin, capacity=10):
        super().__init__(capacity)
        self.r_bin = r_bin
        self.nr_bin = nr_bin

    def receive_garbage(self, garbage, user):
        if self.is_full():
            self.send_warning()
        else:
            self.items.append(garbage)
            self.allocate(garbage, user)
            self.items.pop(0)

    def allocate(self, garbage, user):
        if garbage.sub_type == "recyclable":
            self.r_bin.receive_garbage(garbage)
            user.billing_catalog["recyclable"] += 1
        else:
            self.nr_bin.receive_garbage(garbage)
            user.billing_catalog["non_recyclable"] += 1

    def send_warning(self):
        print("[NB-bin] Warning from NB-bin")


Recyclable & Non-Recyclable Bins

In [9]:
class RBin(Bin):
    def __init__(self):
        super().__init__(capacity=None)

    def receive_garbage(self, garbage):
        self.items.append(garbage)
        print("Recyclable garbage sent to plant")


class NRBin(Bin):
    def __init__(self):
        super().__init__(capacity=None)

    def receive_garbage(self, garbage):
        self.items.append(garbage)
        print("Non-recyclable garbage sent to plant")


Simple Login System

In [10]:
users_db = {}

def register_user(username, password, name, address):
    users_db[username] = User(username, password, name, address)

def login(username, password):
    user = users_db.get(username)
    if user and user.password == password:
        print("Login successful\n")
        return user
    else:
        print("Invalid login")
        return None


System Simulation (Main)

In [13]:
# Register user
register_user("rashed", "1234", "Rashedul Albab", "Sylhet")

# Login
user = login("rashed", "1234")

# Create bins
b_bin = BBin()
r_bin = RBin()
nr_bin = NRBin()
nb_bin = NBBin(r_bin, nr_bin)
gmp_bin = GMPBin(b_bin, nb_bin)
source_bin = SourceBin(user, gmp_bin)

# Add garbage
garbages = [
    Garbage("biodegradable"),
    Garbage("non_biodegradable", "recyclable"),
    Garbage("non_biodegradable", "non_recyclable"),
    Garbage("biodegradable"),
    Garbage("non_biodegradable", "recyclable")
]

for g in garbages:
    source_bin.receive_garbage(g)

# Check bill
user.check_bill()


Login successful

Biodegradable garbage sent to plant
Recyclable garbage sent to plant
Non-recyclable garbage sent to plant
Biodegradable garbage sent to plant
Recyclable garbage sent to plant

--- Billing Summary for Rashedul Albab ---
biodegradable: 2 items -> 0 tk
recyclable: 2 items -> 4 tk
non_recyclable: 1 items -> 5 tk
TOTAL BILL: 9 tk

